In [1]:
import csv

import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers
import tensorflow_model_optimization as tfmot

import features as features_lib
import params
params = params.Params()

In [2]:
# layers
def _batch_norm(name, params):
    def _bn_layer(layer_input):
        return layers.BatchNormalization(
            name=name,
            center=params.batchnorm_center,
            scale=params.batchnorm_scale,
            epsilon=params.batchnorm_epsilon,
        )(layer_input)

    return _bn_layer


def _conv(name, kernel, stride, filters, params):
    def _conv_layer(layer_input):
        output = layers.Conv2D(
            name="{}/conv".format(name),
            filters=filters,
            kernel_size=kernel,
            strides=stride,
            padding=params.conv_padding,
            use_bias=False,
            activation=None,
        )(layer_input)
        output = _batch_norm("{}/conv/bn".format(name), params)(output)
        # output = layers.ReLU(name="{}/relu".format(name))(output)
        output = tf.nn.relu6(output, "{}/relu6".format(name))
        return output

    return _conv_layer


def _separable_conv(name, kernel, stride, filters, params):
    def _separable_conv_layer(layer_input):
        output = layers.DepthwiseConv2D(
            name="{}/depthwise_conv".format(name),
            kernel_size=kernel,
            strides=stride,
            depth_multiplier=1,
            padding=params.conv_padding,
            use_bias=False,
            activation=None,
        )(layer_input)
        output = _batch_norm("{}/depthwise_conv/bn".format(name), params)(output)
        # output = layers.ReLU(name="{}/depthwise_conv/relu".format(name))(output)
        output = tf.nn.relu6(output, "{}/depthwise_conv/relu6".format(name))
        output = layers.Conv2D(
            name="{}/pointwise_conv".format(name),
            filters=filters,
            kernel_size=(1, 1),
            strides=1,
            padding=params.conv_padding,
            use_bias=False,
            activation=None,
        )(output)
        output = _batch_norm("{}/pointwise_conv/bn".format(name), params)(output)
        # output = layers.ReLU(name="{}/pointwise_conv/relu".format(name))(output)
        output = tf.nn.relu6(output, "{}/pointwise_conv/relu6".format(name))
        return output

    return _separable_conv_layer


In [3]:
_YAMNET_LAYER_DEFS = [
    # (layer_function, kernel, stride, num_filters)
    (_conv, [3, 3], 2, 32),
    (_separable_conv, [3, 3], 1, 64),
    (_separable_conv, [3, 3], 2, 128),
    (_separable_conv, [3, 3], 1, 128),
    (_separable_conv, [3, 3], 2, 256),
    (_separable_conv, [3, 3], 1, 256),
    (_separable_conv, [3, 3], 2, 512),
    (_separable_conv, [3, 3], 1, 512),
    (_separable_conv, [3, 3], 1, 512),
    (_separable_conv, [3, 3], 1, 512),
    (_separable_conv, [3, 3], 1, 512),
    (_separable_conv, [3, 3], 1, 512),
    (_separable_conv, [3, 3], 2, 1024),
    (_separable_conv, [3, 3], 1, 1024),
]

In [5]:
# model definition
waveform = layers.Input(
    batch_shape=(params.min_num_samples,),
    dtype=tf.float32,
    name="waveform_binary 0"
)
(
    log_mel_spectrogram,
    features,
) = features_lib.waveform_to_log_mel_spectrogram_patches(waveform, params)

# quant_features = tf.quantization.quantize(
#     features,
#     -2700.0,
#     2700.0,
#     tf.dtypes.quint8,
#     mode='MIN_COMBINED',
#     round_mode='HALF_AWAY_FROM_ZERO',
#     name=None,
#     narrow_range=False,
#     axis=None,
#     ensure_minimum_range=0.01
# )


# features_casted = tf.cast(features, tf.int8)
r1 = net = layers.Reshape(
    (params.patch_frames, params.patch_bands, 1),
    input_shape=(params.patch_frames, params.patch_bands),
)(features)
# )(quant_features)

for (i, (layer_fun, kernel, stride, filters)) in enumerate(_YAMNET_LAYER_DEFS):
    net = layer_fun("layer{}".format(i + 1), kernel, stride, filters, params)(net)
embeddings = layers.GlobalAveragePooling2D()(net)
logits = layers.Dense(units=params.num_classes, use_bias=True)(embeddings)
predictions = layers.Activation(activation=params.classifier_activation)(logits)


In [ ]:
# tfmot quantizers test
tfmot.quantization.keras.quantizers.AllValuesQuantizer(
    8,
    True,
    True,
    
)

In [7]:
yamnet = Model(
    name="yamnet_test",
    inputs=waveform,
    outputs=[predictions]
)
yamnet.load_weights('yamnet.h5')

In [14]:
# try to isolate just the cnn
dummy_input = layers.Input(
    batch_shape=(1, 96, 64),
    dtype=tf.float32,
    name="dummy input"
)
# r1(dummy_input)
cnn = tf.keras.Sequential(
    [
        dummy_input,
        r1
    ],
    # name="yamnet_cnn",
    # inputs=dummy_input,
    # outputs=[logits]
)

TypeError: The added layer must be an instance of class Layer. Found: KerasTensor(type_spec=TensorSpec(shape=(1, 96, 64, 1), dtype=tf.float32, name=None), name='reshape_1/Reshape:0', description="created by layer 'reshape_1'")

In [12]:
# convert to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter = tf.lite.TFLiteConverter.from_keras_model(yamnet)
tflite_model = converter.convert()
# save to file called me/yamnet.tflite
# with open('models/3/me/yamnet_quant.tflite', 'wb') as f:
with open('models/3/me/features.tflite', 'wb') as f:
    f.write(tflite_model)

ConverterError: C:\Users\tonyr\Anaconda3\envs\audio2\lib\site-packages\keras\engine\functional.py:415:0: error: 'tf.QuantizeV2' op is neither a custom op nor a flex op
C:\Users\tonyr\Anaconda3\envs\audio2\lib\site-packages\keras\engine\base_layer.py:1037:0: note: called from
C:\Users\tonyr\Anaconda3\envs\audio2\lib\site-packages\tensorflow\lite\python\tflite_keras_util.py:194:0: note: called from
C:\Users\tonyr\Anaconda3\envs\audio2\lib\site-packages\tensorflow\python\eager\def_function.py:668:0: note: called from
C:\Users\tonyr\Anaconda3\envs\audio2\lib\site-packages\tensorflow\python\framework\func_graph.py:1007:0: note: called from
C:\Users\tonyr\Anaconda3\envs\audio2\lib\site-packages\tensorflow\python\eager\function.py:3308:0: note: called from
C:\Users\tonyr\Anaconda3\envs\audio2\lib\site-packages\tensorflow\python\eager\function.py:3463:0: note: called from
C:\Users\tonyr\Anaconda3\envs\audio2\lib\site-packages\tensorflow\python\eager\function.py:3066:0: note: called from
C:\Use